# 노드 재시도 정책 추가 방법


<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 안내서는 다음에 대한 친숙함을 전제로 합니다:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/">
                    LangGraph 용어집
                </a>
            </li>
        </ul>
    </p>
</div> 


API를 호출하거나, 데이터베이스를 쿼리하거나, LLM을 호출하는 등 노드에 사용자 정의 재시도 정책이 필요할 수 있는 많은 사용 사례가 있습니다. 

## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정하겠습니다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain_anthropic langchain_community


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 개선하세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버그하고, 테스트하고, 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대해 더 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 클릭하세요.
    </p>
</div>


재시도 정책을 구성하기 위해서는 `add_node`에 `retry` 매개변수를 전달해야 합니다. `retry` 매개변수는 `RetryPolicy`라는 명명된 튜플 객체를 사용합니다. 아래에서는 기본 매개변수로 `RetryPolicy` 객체를 인스턴스화합니다:


In [3]:
from langgraph.pregel import RetryPolicy

RetryPolicy()


RetryPolicy(initial_interval=0.5, backoff_factor=2.0, max_interval=128.0, max_attempts=3, jitter=True, retry_on=<function default_retry_on at 0x78b964b89940>)

기본적으로 `retry_on` 매개변수는 `default_retry_on` 함수를 사용하며, 다음의 예외를 제외한 모든 예외에 대해 재시도합니다:

*   `ValueError`
*   `TypeError`
*   `ArithmeticError`
*   `ImportError`
*   `LookupError`
*   `NameError`
*   `SyntaxError`
*   `RuntimeError`
*   `ReferenceError`
*   `StopIteration`
*   `StopAsyncIteration`
*   `OSError`

또한, `requests` 및 `httpx`와 같은 인기 있는 HTTP 요청 라이브러리의 예외에 대해 5xx 상태 코드에서만 재시도합니다.


## 노드에 재시도 정책 전달하기

마지막으로, `add_node` 함수를 호출할 때 `RetryPolicy` 객체를 전달할 수 있습니다. 아래 예시에서는 각 노드에 두 가지 다른 재시도 정책을 전달합니다:


In [5]:
import operator
import sqlite3
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import AIMessage

db = SQLDatabase.from_uri("sqlite:///:memory:")

model = ChatAnthropic(model_name="claude-2.1")


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def query_database(state):
    query_result = db.run("SELECT * FROM Artist LIMIT 10;")
    return {"messages": [AIMessage(content=query_result)]}


def call_model(state):
    response = model.invoke(state["messages"])
    return {"messages": [response]}


# Define a new graph
builder = StateGraph(AgentState)
builder.add_node(
    "query_database",
    query_database,
    retry=RetryPolicy(retry_on=sqlite3.OperationalError),
)
builder.add_node("model", call_model, retry=RetryPolicy(max_attempts=5))
builder.add_edge(START, "model")
builder.add_edge("model", "query_database")
builder.add_edge("query_database", END)

graph = builder.compile()
